# 基于Pytorch的MLP实现
## 目标
- 使用pytorch构建MLP网络
- 训练集使用MNIST数据集
- 使用GPU加速运算
- 要求准确率能达到92%以上
- 保存模型
## 实现
### 数据集：MNIST数据集的载入
MNIST数据集是一种常用的数据集，为28\*28的手写数字训练集，label使用独热码，在pytorch中，可以使用`torchvision.datasets.MNIST()`和`torch.utils.data.DataLoader（）`来导入数据集,其中
- `torchvision.datasets.MNIST()`:用于下载，导入数据集
- `torch.utils.data.DataLoader（）`:用于将数据集整理成batch的形式并转换为可迭代对象

In [24]:
import torch as pt
import torchvision as ptv
import numpy as np

In [15]:
train_set = ptv.datasets.MNIST("../../pytorch_database/mnist/train",train=True,transform=ptv.transforms.ToTensor(),download=True)
test_set = ptv.datasets.MNIST("../../pytorch_database/mnist/test",train=False,transform=ptv.transforms.ToTensor(),download=True)

Processing...
Done!
Processing...
Done!


In [16]:
train_dataset = pt.utils.data.DataLoader(train_set,batch_size=100)
test_dataset = pt.utils.data.DataLoader(test_set,batch_size=100)

### 网络结构构建
网络使用最简单的MLP模型，使用最简单的线性层即可构建,本次网络一共有3层全连接层，分别为28\*28->512,512->128,128->10,除了输出层的激活函数使用softmax以外，其他均采用relu

In [59]:
class MLP(pt.nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.fc1 = pt.nn.Linear(784,512)
        self.fc2 = pt.nn.Linear(512,128)
        self.fc3 = pt.nn.Linear(128,10)
        
    def forward(self,din):
        din = din.view(-1,28*28)
        dout = pt.nn.functional.relu(self.fc1(din))
        dout = pt.nn.functional.relu(self.fc2(dout))
        return pt.nn.functional.softmax(self.fc3(dout))
model = MLP().cuda()
print(model)

MLP (
  (fc1): Linear (784 -> 512)
  (fc2): Linear (512 -> 128)
  (fc3): Linear (128 -> 10)
)


### 代价函数，优化器和准确率检测
代价函数使用交叉熵函数，而考虑pytorch文档中没有检测到argmax函数，决定使用numpy计算准确率，优化器使用最简单的SGD

In [84]:
# loss func and optim
optimizer = pt.optim.SGD(model.parameters(),lr=0.01,momentum=0.9)
lossfunc = pt.nn.CrossEntropyLoss().cuda()

# accuarcy
def AccuarcyCompute(pred,label):
    pred = pred.cpu().data.numpy()
    label = label.cpu().data.numpy()
#     print(pred.shape(),label.shape())
    test_np = (np.argmax(pred,1) == label)
    test_np = np.float32(test_np)
    return np.mean(test_np)

# test accuarcy
# print(AccuarcyCompute(
#     np.array([[1,10,6],[0,2,5]],dtype=np.float32),
#     np.array([[1,2,8],[1,2,5]],dtype=np.float32)))

### 训练网络
训练网络的步骤分为以下几步：
1. 初始化，清空网络内上一次训练得到的梯度
2. 载入数据为Variable，送入网络进行前向传播
3. 计算代价函数，并进行反向传播计算梯度
4. 调用优化器进行优化

In [88]:
for x in range(2):
    for i,data in enumerate(train_dataset):
    
        optimizer.zero_grad()
    
        (inputs,labels) = data
        inputs = pt.autograd.Variable(inputs).cuda()
        labels = pt.autograd.Variable(labels).cuda()
    
        outputs = model(inputs)
    
        loss = lossfunc(outputs,labels)
        loss.backward()
    
        optimizer.step()
    
        if i % 100 == 0:
            print(i,":",AccuarcyCompute(outputs,labels))

0 : 0.95
100 : 0.92
200 : 0.91
300 : 0.91
400 : 0.93
500 : 0.91
0 : 0.95
100 : 0.92
200 : 0.93
300 : 0.91
400 : 0.93
500 : 0.9
